# Assignment 05
EE 538 Spring 2020 <br>
Analog Circuits for Sensor Systems <br>
University of Washington Electrical & Computer Engineering <br>

Due: May 9, 2020 <br>
Author: Kevin Egedy

In [2]:
# Imports
import os
import sys
import cmath
import math
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import ltspice
import sympy as sp
from scipy import signal
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
def read_ltspice(file_name,ftype='trans'):
    cols = []
    arrs = []
    with open(file_name, 'r',encoding='utf-8') as data:
        for i,line in enumerate(data):
            if i==0:
                cols = line.split()
                arrs = [[] for _ in cols]
                continue
            parts = line.split()
            for j,part in enumerate(parts):
                arrs[j].append(part)
    df = pd.DataFrame(arrs,dtype='float64')
    df = df.T
    df.columns = cols
    if ftype=='trans':
        return df
    elif ftype=='ac':
        for col in cols:
            if df[col].str.contains(',').all():
                df['Mag'] = df[col].apply(lambda x: x.split(',')[0])
                df['Mag'] = df['Mag'].apply(lambda x: x[1:-2])
                df['Mag'] = df['Mag'].astype('float64')
                df['Phase'] = df[col].apply(lambda x: x.split(',')[1])
                df['Phase'] = df['Phase'].apply(lambda x: x[0:-2])
                df['Phase'] = df['Phase'].astype('float64')
        return df
    else:
        print('invalid ftype')

<img src="images/HW05Q1.png" width="600">

**Differential Amplifier Stage**

$\begin{eqnarray}
&& Z_{\text{in}} &=& \frac{\Delta V_{id}}{\Delta(I_{-}-I^{+})}&=&
R_i(1+\frac{R_i}{R_f})
\end{eqnarray}$

<img src="images/HW05Q2.png" width="600">

**Instrumentation Amplifier Stage**

$\begin{eqnarray}
&& A_{v_{d1}} &=& \frac{V_{o_d}}{V_{i_d}} &=& 1 + \frac{R_{f_p}+R_{f_m}}{R_G} \\
&& A_{v_{cm1}} &=& \frac{V_0}{V_{i_{cm}}} &=& 1 \\
&& \text{CMMR} &=& \frac{A_{v_{d1}}}{A_{v_{cm1}}} &=& 1 + \frac{R_f}{R_G}
\end{eqnarray}$